In [7]:
import pandas as pd
import numpy as np
from skmultilearn.model_selection import IterativeStratification

TRAIN_CSV = 'D:/HPA_comp/single_cells/train.csv'

df = pd.read_csv(TRAIN_CSV, index_col=0)
# Shuffle df to mix up single cell images before stratification
df = df.sample(frac=1, random_state=42)
df.reset_index(drop=True, inplace=True)

In [9]:
def extract_as_array(str_):
            list_ = str_.strip('][').split(', ')
            return np.array([int(i) for i in list_])
images = df['cell_id'].values
targets = df['Label'].apply(extract_as_array).values

# Reshape to single arrays
img_mat = images.reshape(-1, 1)
target_mat = np.vstack(targets)

In [20]:
k_fold = IterativeStratification(n_splits=5, order=1)
splits = k_fold.split(img_mat, target_mat)

df['fold'] = np.nan  # Generate folds column

for fold, (_, fold_idxs) in enumerate(splits):
    valid_imgs = img_mat[fold_idxs]
    df.loc[df['cell_id'].isin(valid_imgs.reshape(-1)), 'fold'] = fold

In [21]:
OUT_CSV = 'D:/HPA_comp/single_cells/train_folds.csv'
#df.to_csv(OUT_CSV)
df['fold'].value_counts()

0.0    23
1.0    23
3.0    22
2.0    22
4.0    22
Name: fold, dtype: int64